# Data wrangling
This notebooks is used to import and clean the datasets

In [ ]:
DATA_FOLDER = './data/'
MOVIES_FOLDER = DATA_FOLDER + 'movies_summaries/'
PLOT_SUMMARY_FOLDER = DATA_FOLDER + 'corenlp_plot_summaries'
KAGGLE_FOLDER = DATA_FOLDER + 'kaggle/'

REPORT_FOLDER = './gen/reports/'
ETHNICITY_FILE = './gen/ethnicities.tsv'

CHARACTERS_FILE = MOVIES_FOLDER + 'character.metadata.tsv'
MOVIES_FILE = MOVIES_FOLDER + 'movie.metadata.tsv'
PLOT_SUMMARIES_FILE = MOVIES_FOLDER + 'plot_summaries.txt'
TROPES_FILE = MOVIES_FOLDER + 'tvtropes.clusters.txt'
KAGGLE_FILE = KAGGLE_FOLDER + 'movies_metadata.csv'

: 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast

# from pandas_profiling import ProfileReport
%matplotlib inline

from data_wrangling_tools import *

: 

# Characters

In [ ]:
# load characters
characters = load_characters(CHARACTERS_FILE)
print(characters.shape)
characters.head()

In [ ]:
# generate profiling report
# profile_characters = ProfileReport(characters, title='Characters report')
# profile_characters.to_file(REPORT_FOLDER + "character_report_0.html")

# Ethnicities

In [ ]:
# load ethnicities
# lines with more than 1 value (to run the script, only the first value has been kept)): 
# /m/03ttfc	Spaniards	4	1
# /m/04y8_bm	Multiracial American	2	3	4	5
# /m/017sq0	Eurasian	3	1
# /m/04zjjt	history of the Jews in India	1	3
# /m/027vy0s	White Latin American	1	4
# /m/01kb9y	multiracial people	2	3	4	5
# /m/04fh1b	Blasians	2	3
# /m/03hjx6f	Black Hispanic and Latino Americans	2	3

# clusters:
ethnicity_clusters = {1: 'White', 2: 'Black', 3: 'Asian', 4: 'Latino', 5: 'Native / Indigenous people'}

ethnicities = load_ethnicities(ETHNICITY_FILE, ethnicity_clusters)
ethnicities.head()

In [ ]:
# add characters ethnicities
characters = add_characters_ethnicities(characters, ethnicities)
characters.head()

# Movies

In [ ]:
# load movies
movies = load_movies(MOVIES_FILE)

# clean features    
movies = clean_unknowns(movies)
movies = clean_jsons(movies)

print(movies.shape)
movies.head()

In [ ]:
# generate profiling report
# profile_movies = ProfileReport(movies, title='Movies report')
# profile_movies.to_file(REPORT_FOLDER + "movies_report_0.html")

In [ ]:
# number of nan box office values
nb_nan_movies = movies['box_office_revenue'].isna().sum()
print('Number of known box office values: {} ({:.2%})'.format(movies.shape[0] - nb_nan_movies, (movies.shape[0] - nb_nan_movies) / movies.shape[0]))
print('Number of nan box office values: {} ({:.2%})'.format(nb_nan_movies, nb_nan_movies / movies.shape[0]))

# kaggle movies

In [ ]:
# import kaggle dataset
def load_kaggle(KAGGLE_file, columns=['original_title', 'revenue', 'budget', 'vote_average', 'vote_count', 'release_date']):
    #kaggle = pd.read_csv(KAGGLE_file, usecols=columns)
    kaggle = pd.read_csv(KAGGLE_file, usecols=columns)

    # remove wrongly formatted rows (only 3)
    kaggle = kaggle.drop(kaggle[kaggle['budget'].str.contains('.jpg')].index)
    kaggle['release_date'] = pd.to_datetime(kaggle['release_date'], format='%Y-%m-%d', errors='coerce')

    # convert numerical columns to float
    kaggle['revenue'] = kaggle['revenue'].astype(float).apply(lambda x: np.nan if x == 0.0 else x)
    kaggle['budget'] = kaggle['budget'].astype(float).apply(lambda x: np.nan if x == 0.0 else x)

    return kaggle

kaggle = load_kaggle(KAGGLE_FILE)
print(kaggle.shape)
kaggle.head()

# Check whether the kaggle dataset can be trusted
(i.e. contains box office values that we can trust that are the same as the ones in the movies dataset)

In [ ]:
df_trust = pd.merge(movies, kaggle, left_on=[movies['name'], movies['release_date'].dt.year], 
                    right_on=[kaggle['original_title'], kaggle['release_date'].dt.year], how='left')
df_trust = df_trust.rename({'release_date_x': 'release_date', 'revenue':'box_office_revenue_kaggle'}, axis=1)
df_trust = df_trust.drop(columns=['original_title', 'key_0', 'key_1', 'release_date_y'])

# select only the rows where the box office revenue is known
df_trust = df_trust[(df_trust['box_office_revenue'].notna()) & (df_trust['box_office_revenue_kaggle'].notna())]

df_trust['box_office_revenue'] = df_trust['box_office_revenue'].astype(float)
df_trust['box_office_revenue_kaggle'] = df_trust['box_office_revenue'].astype(float)

# compare all values
is_same = np.allclose(df_trust['box_office_revenue'].to_numpy(), df_trust['box_office_revenue_kaggle'].to_numpy())

print("Are the values from Kaggle and the values from the movies dataset are the same? {}".format(is_same))

    

# Merge kaggle movies and movies

In [ ]:
# merge movies and kaggle movies

def merge_movies_kaggle(movies, kaggle):

    df = pd.merge(movies, kaggle, left_on=[movies['name'], movies['release_date'].dt.year], 
        right_on=[kaggle['original_title'], kaggle['release_date'].dt.year], how='left')
    df = df.rename({'release_date_x': 'release_date'}, axis=1)

    # fill the box_office revenue with the kaggle revenue if it's missing
    df['box_office_revenue'] = df['box_office_revenue'].fillna(df['revenue'].copy())
    df = df.drop(columns=['revenue', 'original_title', 'key_0', 'key_1', 'release_date_y'])
    
    return df

merged_movies = merge_movies_kaggle(movies, kaggle)

print(merged_movies.shape)
merged_movies.head()

In [ ]:
# number of nan box office values
nb_nan_merged_movies = merged_movies['box_office_revenue'].isna().sum()
print('Number of known box office values: {} ({:.2%})'.format(merged_movies.shape[0] - nb_nan_merged_movies, (merged_movies.shape[0] - nb_nan_merged_movies) / merged_movies.shape[0]))
print('Number of nan box office values: {} ({:.2%})'.format(nb_nan_movies, nb_nan_movies / movies.shape[0]))

nb_non_nan_movies = movies.shape[0] - nb_nan_movies
nb_non_nan_merged_movies = merged_movies.shape[0] - nb_nan_merged_movies
print('Number of box office values gained with kaggle: {} ({:.2%})'.format(nb_non_nan_merged_movies - nb_non_nan_movies, (nb_non_nan_merged_movies - nb_non_nan_movies) / nb_non_nan_movies))

# Merge characters and movies

In [ ]:
# merge characters and movies
df = merge_characters_movies(characters, merged_movies)
df.head()

In [ ]:
# generate clean df
df1 = generate_clean_df_with_kaggle(CHARACTERS_FILE, ETHNICITY_FILE, MOVIES_FILE, KAGGLE_FILE)

assert(df.equals(df1))
df1.head()